### Exploration

In [1]:
!pip install stable-baselines
!pip install --upgrade tensorflow==1.14.0
!pip install gym

Requirement already up-to-date: tensorflow==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)


In [2]:
import os
import time
import gym
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from stable_baselines import deepq
from stable_baselines import bench
from stable_baselines import logger
import tensorflow as tf

from baselines.common.tf_util import make_session

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [3]:
data = pd.read_csv("/project/datasets/clean-ids-collection/cic-ddos2019/clean/cicddos2019.csv")
data.head(5)

,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,2018-11-03 09:18:16.964447,114456999,45,0,0.0,0.0,0.0,0.0,0.000000,...,0,8185.5835,28337.111,98168.0,3.0,9529897.0,351582.62,10001143.0,9048097.0,Benign
1,0,2018-11-03 09:18:18.506537,114347504,56,0,0.0,0.0,0.0,0.0,0.000000,...,0,35028.4180,121314.914,420255.0,4.0,9493930.0,351541.10,9978130.0,8820294.0,Benign
2,6,2018-11-03 09:18:18.610576,36435473,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62416.0000,0.000,62416.0,62416.0,36373056.0,0.00,36373056.0,36373056.0,Benign
3,6,2018-11-03 09:18:18.610579,36434705,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62413.0000,0.000,62413.0,62413.0,36372292.0,0.00,36372292.0,36372292.0,Benign
4,6,2018-11-03 09:18:18.610581,36434626,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62409.0000,0.000,62409.0,62409.0,36372216.0,0.00,36372216.0,36372216.0,Benign


In [4]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138007 entries, 0 to 138006
Data columns (total 69 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Protocol                  138007 non-null  int64  
 1   Timestamp                 138007 non-null  object 
 2   Flow Duration             138007 non-null  int64  
 3   Total Fwd Packets         138007 non-null  int64  
 4   Total Backward Packets    138007 non-null  int64  
 5   Fwd Packets Length Total  138007 non-null  float64
 6   Bwd Packets Length Total  138007 non-null  float64
 7   Fwd Packet Length Max     138007 non-null  float64
 8   Fwd Packet Length Min     138007 non-null  float64
 9   Fwd Packet Length Mean    138007 non-null  float64
 10  Fwd Packet Length Std     138007 non-null  float64
 11  Bwd Packet Length Max     138007 non-null  float64
 12  Bwd Packet Length Min     138007 non-null  float64
 13  Bwd Packet Length Mean    138007 non-null  f

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Act Data Packets,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,138007.000000,1.380070e+05,138007.000000,138007.000000,1.380070e+05,1.380070e+05,138007.000000,138007.000000,138007.000000,138007.000000,...,138007.000000,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05
mean,11.592050,1.035663e+07,13.812169,6.636576,4.019830e+03,5.102600e+03,249.273218,132.853515,161.376744,37.345936,...,10.525227,-1.175840e+07,7.345821e+04,3.066152e+04,1.140858e+05,5.479838e+04,3.445620e+06,3.282404e+05,3.770891e+06,3.153606e+06
std,5.607782,2.853154e+07,70.298079,99.503208,5.151998e+04,1.880790e+05,419.156188,234.493611,241.043532,116.216344,...,62.973954,1.112129e+08,5.176193e+05,2.392317e+05,6.900226e+05,4.802156e+05,1.157949e+07,2.158705e+06,1.230392e+07,1.120812e+07
min,0.000000,1.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,-1.408238e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000,1.178000e+03,2.000000,0.000000,1.800000e+01,0.000000e+00,6.000000,0.000000,6.000000,0.000000,...,1.000000,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,17.000000,2.489200e+04,3.000000,2.000000,8.600000e+01,0.000000e+00,43.000000,30.000000,39.000000,0.000000,...,1.000000,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,17.000000,2.999380e+06,8.000000,2.000000,2.064000e+03,1.660000e+02,440.000000,152.000000,349.500000,14.433757,...,5.000000,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,17.000000,1.199987e+08,20444.000000,31700.000000,1.526642e+07,5.842950e+07,32120.000000,2131.000000,3015.290500,2221.556200,...,18766.000000,1.480000e+03,4.050800e+07,1.100562e+07,4.050800e+07,4.050800e+07,1.192194e+08,6.961402e+07,1.192194e+08,1.192194e+08


Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [5]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    cols = df.columns[:-1]
    vc = df[col].value_counts()
    n = vc.iloc[-1]
    m = vc.iloc[0]
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    df = df.drop(index=initial_cut.index)

    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [6]:
data.drop('Timestamp', inplace=True, axis=1)
print(data['Label'].value_counts())
data['Label'] = data['Label'].astype('object')
atk_idx = data.loc[data['Label'] != "Benign"].index
data.loc[atk_idx, 'Label'] = 1.0
data.loc[data.index.difference(atk_idx), 'Label'] = 0.0
data['Label'] = data['Label'].astype(dtype=np.float32)

x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(data)

Benign           95095
DrDoS_NTP        13687
TFTP             13027
Syn               6713
UDP               2158
MSSQL             1763
DrDoS_UDP         1174
UDP-lag           1097
DrDoS_MSSQL        927
DrDoS_DNS          552
LDAP               484
DrDoS_SSDP         413
DrDoS_SNMP         392
Portmap            215
DrDoS_LDAP         174
NetBIOS             73
DrDoS_NetBIOS       51
UDPLag              10
WebDDoS              2
Name: Label, dtype: int64


In [7]:
print("Training")
print(x_train.shape)
print(y_train.shape)
print("Validation")
print(x_val.shape)
print(y_val.shape)
print("Testing")
print(x_test.shape)
print(y_test.shape)


Training
(17164, 67)
(17164,)
Validation
(8582, 67)
(8582,)
Testing
(60076, 67)
(60076,)


In [8]:
# all attacks
# attacks_dict = {
#     'Benign': 0,
#     'Portmap': 1,
#     'NetBIOS': 2,
#     'LDAP': 3,
#     'MSSQL': 4,
#     'UDP': 5,
#     'UDPLag': 6,
#     'Syn': 7,
#     'DrDoS_NTP': 8,
#     'DrDoS_DNS': 9,
#     'DrDoS_LDAP': 10,
#     'DrDoS_MSSQL': 11,
#     'DrDoS_NetBIOS': 12,
#     'DrDoS_SNMP': 13,
#     'DrDoS_SSDP': 14,
#     'DrDoS_UDP': 15,
#     'UDP-lag': 16,
#     'WebDDoS': 17,
#     'TFTP': 18,
# }


In [9]:
# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)
# x_val = np.expand_dims(x_val, -1)

In [10]:
# y_train = np.expand_dims(y_train, -1)
# y_test = np.expand_dims(y_test, -1)

In [11]:
print(x_train.shape)

(17164, 67)


In [12]:
print(x_train.index)
print(y_train.index)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

Int64Index([ 93941, 134014,  40695,  37947, 125172,  92148, 134349,  40774,
             40327,  57948,
            ...
            112066,  23546,  73787,  98313, 119797,  79607,  71518,  28683,
            115377,  93729],
           dtype='int64', length=17164)
Int64Index([ 93941, 134014,  40695,  37947, 125172,  92148, 134349,  40774,
             40327,  57948,
            ...
            112066,  23546,  73787,  98313, 119797,  79607,  71518,  28683,
            115377,  93729],
           dtype='int64', length=17164)


In [13]:
# custom keys -> replace by index

x_train = x_train.set_index([pd.Index(range (0, len(x_train)))])
y_train = y_train.set_index([pd.Index(range (0, len(y_train)))])
x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])


In [14]:
print(x_train.index)
print(y_train.index)

RangeIndex(start=0, stop=17164, step=1)
RangeIndex(start=0, stop=17164, step=1)


In [15]:
print(y_train)

       Label
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
...      ...
17159    1.0
17160    1.0
17161    1.0
17162    1.0
17163    1.0

[17164 rows x 1 columns]


In [16]:
print(y_train.iloc[17162,:])

Label    1.0
Name: 17162, dtype: float32


In [17]:
print(y_train.index)
print(x_train.index)

RangeIndex(start=0, stop=17164, step=1)
RangeIndex(start=0, stop=17164, step=1)


### Custom environment

In [18]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        # change to 19 if detectiong all different attacks
        self.action_space = gym.spaces.Discrete(2)
        # All the features we have, 67 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(67,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

Train classifier using dqn

In [19]:
def ids_dqn():
#     logger.configure(dir=os.path.join(os.getcwd(),'Training/Logs/ids_dqn'), format_strs=['stdout', 'tensorboard'])
    env = IdsEnv(images_per_episode=4096)
#     env = bench.Monitor(env, logger.get_dir())

    model = deepq.DQN(
        MlpPolicy, 
        env, 
        verbose=1, 
        learning_rate=0.00025, 
        buffer_size=1000000, 
        learning_starts=50000, 
        batch_size=32, 
        gamma=0.99, 
        train_freq=4, 
        target_network_update_freq=10000, # 10k to 30k
        exploration_fraction=0.1, # 0.1 to 0.01
        exploration_initial_eps=1.0, 
        exploration_final_eps=0.05, 
    ).learn(
        total_timesteps=int(1.0e7),
        log_interval=1
    )

    model.save('ddqn_stableb2_ids.pkl')
    env.close()
    
    return model

start_time = time.time()
dqn_model = ids_dqn()
print("DQN Training Time:", time.time() - start_time)








Instructions for updating:
Use keras.layers.flatten instead.


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


--------------------------------------
| % time spent exploring  | 99       |
| episodes                | 2        |
| mean 100 episode reward | 2.09e+03 |
| steps                   | 4096     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 99       |
| episodes                | 3        |
| mean 100 episode reward | 2.08e+03 |
| steps                   | 8192     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 98       |
| episodes                | 4        |
| mean 100 episode reward | 2.07e+03 |
| steps                   | 12288    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 98       |
| episodes                | 5        |
| mean 100 episode reward | 2.06e+03 |
| steps                   | 16384    |
-------

In [20]:
def ids_dqn_eval(dqn_model):
    attempts, correct = 0,0

    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, _ = env.step(dqn_model.predict(obs)[0])

                attempts += 1
                if rew > 0:
                    correct += 1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

ids_dqn_eval(dqn_model)

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 67.34028032093751%


##### 